# Imports and Dependencies

In [15]:
# Imports
import pandas as pd
import math as m

# Reads in csv files. Begins formatting.
nfa = pd.read_csv("resources/NFA_Data.csv")
un = pd.read_csv("resources/UN_Data.csv", header=4).drop([0,67,121,159,193])

C:\Users\mrjac\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# NFA Data

In [5]:
# Creating NFA dataframe for 1990, 2000, and 2010.
nfa = nfa[['country','year','carbon','record','QScore']]
nfa_data1990 = nfa[nfa['year'] == 1990]
nfa_data2000 = nfa[nfa['year'] == 2000]
nfa_data2010 = nfa[nfa['year'] == 2010]
nfa = ((nfa_data1990.append(nfa_data2000)).append(nfa_data2010))

# Dropping world summary data and resetting index values.
nfa = nfa[nfa['country'] != 'World']
nfa = nfa.set_index("country").reset_index()

In [6]:
# Narrows dataframe to only include relevant carbon rows (production, consumption).
nfa = nfa[(nfa['record'] == 'EFProdPerCap') | (nfa['record'] == 'EFConsPerCap')]

# Drops NaN values and resets index values.
nfa.dropna(how='any',inplace=True)
nfa = nfa.set_index("country").reset_index()

In [8]:
# Reformats NFA data and overwrites 'nfa' with new dataframe for merging.
i = 0
country_col = []
year_col = []
c_cons_col = []
c_prods_col = []
q_col = []

# For loop to pull dataframe values.
for row in nfa['country']:
    if i < len(nfa['country']):
        country_col.append(nfa['country'][i])
        year_col.append(nfa['year'][i])
        c_cons_col.append(nfa['carbon'][i])
        q_col.append(nfa['QScore'][i])
        i += 1
        c_prods_col.append(nfa['carbon'][i])
        i += 1

# Creates dataframe for merging.
nfa = pd.DataFrame()
nfa['country'] = country_col
nfa['year'] = year_col
nfa['c_consumed'] = c_cons_col
nfa['c_produced'] = c_prods_col
nfa['QScore'] = q_col

# UN Data

In [9]:
# Selects only columns with valid, single year names from UN dataset.
selected_columns = []
for column in un:
    if 'Unnamed' not in column:
        if '-' not in column:
            selected_columns.append(column)

# Turns '..' into NaN values. Drops rows with only NaN values.
un = un.replace("..","NaN") 
un = (un[selected_columns])
un.dropna(thresh = 9, inplace = True)

# Drops summary data rows.
un.drop(un.tail(36).index,inplace=True)

# Transforms data from horizontal to vertical.
un = pd.melt(un, id_vars = 'Country', value_vars = ['1990','2000','2010', '2014', '2015'])

# Renames columns.
un = un.rename(columns = {'Country':'country','variable':'year','value':'hdi'})

# Final Dataset

In [10]:
# Retypes country and year columns to allow for clean merging.
un['country'] = un['country'].astype('string')
nfa['country'] = nfa['country'].astype('string')
un['year'] = un['year'].astype('int64')
nfa['year'] = nfa['year'].astype('int64')

In [11]:
# Merges on country and year values for NFA and UN datasets.
data = nfa.merge(un, on = ['country','year'])

In [14]:
# Exports final dataframe to csv for pgAdmin.
data.to_csv('resources/Final_Dataset.csv')